In [1]:
import pandas as pd
import numpy as np
import regex
import string
import matplotlib.pyplot as plt
from analysis.analyzer import TTTH_Analyzer as Analyzer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


def get_top_3_similar_products(ma_sp, sp, corr_mat):
    # Tìm các sản phẩm chung hệ, đảm bảo là danh sách phẳng (flat list)
    sp_cung_he = sp[sp['ma_san_pham'] == ma_sp]['san_pham_cung_he'].tolist()[0]  # Lấy danh sách sản phẩm tương ứng

    # Tìm kiếm dòng có ma_san_pham = ma_sp và trả về index
    index_can_tim = sp[sp['ma_san_pham'] == ma_sp].index.tolist()[0]

    # Tạo result_df từ ma trận tương tự
    result_df = pd.DataFrame(corr_mat.iloc[index_can_tim])
    result_df.rename(columns={result_df.columns[0]: 'similarity'}, inplace=True)

    # Tạo ánh xạ ma_san_pham và diem_trung_binh
    mapping = dict(enumerate(sp['ma_san_pham']))
    diem_trung_binh_mapping = dict(enumerate(sp['diem_trung_binh']))
    sp_cung_he_mapping = dict(enumerate(sp['san_pham_cung_he']))
    gia_ban_mapping = dict(enumerate(sp['gia_ban']))

    # Ánh xạ ma_san_pham và diem_trung_binh vào result_df theo index
    result_df['ma_san_pham'] = result_df.index.map(mapping)
    result_df['diem_trung_binh'] = result_df.index.map(diem_trung_binh_mapping)
    result_df['san_pham_cung_he'] = result_df.index.map(sp_cung_he_mapping).astype('str')
    result_df['gia_ban'] = result_df.index.map(gia_ban_mapping)

    # Sắp xếp theo 'san_pham_cung_he', 'diem_trung_binh' (giảm dần), và 'gia_ban' (tăng dần)
    result_df = result_df.sort_values(by=['san_pham_cung_he', 'diem_trung_binh', 'gia_ban'], ascending=[True, False, True])

    # Lọc bỏ các dòng trùng 'san_pham_cung_he', giữ lại dòng đầu tiên trong mỗi nhóm
    result_df = result_df.drop_duplicates(subset='san_pham_cung_he', keep='first')

    # Sử dụng apply để kiểm tra sự tồn tại của 'ma_san_pham' trong 'san_pham_cung_he'
    top3_similarity = result_df[(result_df['diem_trung_binh'] >= 4) & (~result_df['ma_san_pham'].apply(lambda x: str(x) in sp_cung_he)) & (result_df['similarity'] < 0.9)].sort_values(by='similarity', ascending=False).head(3)

    # Thêm sản phẩm của ma_sp vào kết quả
    top3_similarity = pd.concat([
        sp[sp['ma_san_pham'] == ma_sp][['ma_san_pham']],
        top3_similarity
    ], ignore_index=True)

    # Trả về kết quả sau khi merge với thông tin sản phẩm
    result = pd.merge(top3_similarity, sp[['ma_san_pham', 'ten_san_pham', 'mo_ta', 'gia_goc', 'phan_loai']], on='ma_san_pham', how='left')

    return result


In [2]:
sp = pd.read_csv('All_San_pham_clean.csv',sep=';')
cosine_matrix = np.load('cosine_similarity.npy')
df_cosine = pd.DataFrame(cosine_matrix)

In [3]:
# Ví dụ sử dụng hàm
ma_sp = 318900011
result_df = get_top_3_similar_products(ma_sp, sp, df_cosine)
# result_df.style.applymap(lambda x: 'background-color: yellow' if x == ma_sp else '', subset=['ma_san_pham'])
result_df

,ma_san_pham,similarity,diem_trung_binh,san_pham_cung_he,gia_ban,ten_san_pham,mo_ta,gia_goc,phan_loai
0,318900011,NaN,NaN,NaN,NaN,Nước Hoa Hồng Klairs Dành Cho Da Nhạy Cảm 180ml,Nước Hoa Hồng Klairs Supple Preparation là dòn...,409000.0,2x180ml 180ml Không Mùi Có Mùi Hương
1,318900002,0.444821,4.5,[318900002],275000.0,Serum Klairs Vitamin C Cho Da Nhạy Cảm 35ml,"Tinh Chất Klairs Vitamin C Dưỡng Sáng Da, Mờ T...",455000.0,NaN
2,318900017,0.344559,5.0,[318900017],42000.0,Mặt Nạ Klairs Làm Sạch & Dịu Mát Da 25ml,Mặt Nạ Klairs Hỗ Trợ Làm Sạch & Dịu Mát Da 25m...,64000.0,1 miếng 2 miếng
3,318900003,0.293168,4.8,[318900003],297000.0,Dầu Tẩy Trang Klairs Làm Sạch Sâu Cho Mọi Loại...,Dầu Tẩy Trang Klairs Làm Sạch Sâu Cho Mọi Loại...,425000.0,NaN
